In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [619]:
%pip install -U langchain langchain_openai langchain_experimental langgraph duckduckgo-search sentence-transformers surrealdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 2.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.tools import DuckDuckGoSearchRun
tools = [DuckDuckGoSearchRun()]

In [3]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

In [7]:
from langchain_openai import ChatOpenAI

# We will set streaming=True so that we can stream tokens
# See the streaming section for more information on this.

model = ChatOpenAI(
    api_key='ollama',
    base_url='http://localhost:11434/v1',
    model='mistral',
    temperature=0, 
    streaming=True)

In [8]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

In [9]:
from langchain_community.vectorstores import SurrealDBStore
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings()
sdb = SurrealDBStore(embedding_function=embedding_function, db_user="root", db_pass="root")

await sdb.initialize()

/home/karim/Jupyter/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [629]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

documents = TextLoader("state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

await sdb.adelete()
await sdb.aadd_documents(docs)

['documents:h96ejbnpjqnncvidboxo',
 'documents:o5w8x44oiq81u9v5u4xo',
 'documents:k65f3pg6z86fqg52xiv6',
 'documents:0285ks1u3z8sa54l95mp',
 'documents:swr0qhi1sqiyfqfm07pc',
 'documents:sqwr8q3fl2fbyg8eqz97',
 'documents:343xknpmdw67s4p406di',
 'documents:qkzezz7c4gxoy64ybl0s',
 'documents:3yfo3l4vi55qvdkl990q',
 'documents:4jl8nvhig022v6w6mc36',
 'documents:o09xdj9b17kq2xgb3who',
 'documents:6tlws6mcqmff2b7bukwz',
 'documents:q2z2bwhr3ilthvd37dhb',
 'documents:9forwkkqj1e03wc6g5o0',
 'documents:rxxwy95a728s1okhsvmv',
 'documents:gqt6pti397p6pdz9mw51',
 'documents:1btjx002kpd234x9c3sf',
 'documents:k3msjjjw6q7rc6ovbgg9',
 'documents:mfdqt3zlsyjgdnlv63qj',
 'documents:rk94mnyg22gkq3yi2rbp',
 'documents:sal0awa7a17n8lg804qi',
 'documents:9zorkadxixa8dut015t1',
 'documents:k0i5tastg5kcmkqn1fwp',
 'documents:0xn0f7g32nrz4v08s82n',
 'documents:upcvxevum53lcwnm7pme',
 'documents:dm4hpw49f33bvtsndr6m',
 'documents:uo0bbb55de2lqz7bmn2q',
 'documents:i1v2do7eth3tjft4dajn',
 'documents:t8gm2ngw

In [10]:
query = "What did the president say about Ketanji Brown Jackson"
await sdb.asimilarity_search(query)

[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'id': 'documents:3tgwcm7wbtsexlho9x7n', 'source': 'state_of_the_union.txt'}),
 Document(page_content='As Frances Haugen, who is here with us tonight, has shown, we must h

In [11]:
await sdb.asimilarity_search_with_relevance_scores(query)

[(Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'id': 'documents:3tgwcm7wbtsexlho9x7n', 'source': 'state_of_the_union.txt'}),
  0.3983954794333666),
 (Document(page_content='As Frances Haugen, who is here with us toni

In [106]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate

main_prompt = PromptTemplate.from_template(
    """You are a helpful assistant. Provide a response based on the below context. 
    If you don't have an answer, just say "I don't know". Don't suggest an answer.

    Context:
    {context}

    Response:
    """
)

main_chain = main_prompt | model

rag_prompt = PromptTemplate.from_template(
    """You are a helpful assistant. Provide a response based on the below context. 
    If you don't have an answer, just say "I don't know". Don't suggest an answer.

    Context:
    {context}

    Human: {question}
    AI: """
)

rag_chain = (
    {
        "context": sdb.as_retriever(),
        "question": RunnablePassthrough()
    } | rag_prompt | model
)

review_prompt = PromptTemplate.from_template(
    """Review Message 1 and Message 2 below. Prepare a response accordingly. See reference examples below:

    Only the below are valid responses:
    1. If questions is about the state of the union, or asks what the President of United States say, then prepare a json response with the following format:

    {{"retrieve": true}}

    2. If Message 2 includes a satisfactory answer for the question, and doesn't include phrase similar to "I cannot provide.." etc, then prepare a json response with the following format:

    {{"response": true}}
    
    3. If neither 1 or 2 work, then prepare a json response with the following format:

    {{"function": {{"arguments": {{"query": "question"}}, "name": "duckduckgo_search"}}}}
    
    We only have access to "duckduckgo_search".
    Do not include any additional explanation.
    Do not respond that you can not provide an answer.
    
    Examples:

    Message 1: What did the president say in the third state of the union?
    Message 2: Some response
    Final Response: {{"retrieve": true}}
    
    Message 1: What is the capital of France?
    Message 2: Capital of France is Paris.
    Final Response: {{"response": true}}

    Message 1: Summarize the state of the union
    Message 2: summary of the state of the union
    Final Response: {{"retrieve": true}}

    Message 1: Why is the sky blue?
    Message 2: I don't have an answer for this question. I don't have real-time access to that data.
    Final Response: {{"function": {{"arguments": {{"query": "Why is the sky blue?"}}, "name": "duckduckgo_search"}}}}

    Message 1: What is 2+2?
    Message 2: 2 + 2 = 4
    Final Response: {{"response": true}}

    Message 1: Who is Motoko Kusinagi?
    Message 2: I don't have an answer for this question. I don't have real-time access to that data.
    Final Response: {{"function": {{"arguments": {{"query": "Who is Motoko Kusinagi?"}}, "name": "duckduckgo_search"}}}}

    It is very important that you only respond with valid json and nothing else.
    
    Message 1: {message_1}
    Message 2: {message_2}
    Final Response: 
    """
)

review = review_prompt | model

In [107]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [108]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def router(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function" in last_message.additional_kwargs:
        return "continue"
    # Otherwise if there is, we continue
    else:
        if "retrieve" in last_message.additional_kwargs:
            return "retrieve"
        else:
            return "end"

def review_model(state):
    messages = state['messages']
    response = messages[-1]
    request = [message for message in messages if isinstance(message,HumanMessage)][-1]

    question = request.content
    answer = response.content

    review_response = review.invoke({"message_1":question,"message_2":answer})

    try:
        resp = json.loads(review_response.content)
        if "function" in resp:
            resp["function"]["arguments"] = json.dumps(resp["function"]["arguments"])
            response.additional_kwargs = resp
            response.content = ""
        else:
            if "retrieve" in resp:
                response = request
                response.additional_kwargs = {"retrieve":True}
    except:
        pass
    # """
    # We return a list, because this will get added to the existing list
    print(response)
    return {"messages": [response]}
    
# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    
    # response = model.invoke(messages)
    response = main_chain.invoke({"context":messages})
    
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

async def call_rag(state):
    messages = state['messages']

    last_message = messages[-1]

    question = last_message.content
    response = await rag_chain.ainvoke(question)

    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function"]["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [109]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("rag", call_rag)
workflow.add_node("review", review_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# After 'agent' is called, 'review' is called next
workflow.add_edge("agent","review")
workflow.add_edge("rag","review")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "review",
    # Next, we pass in the function that will determine which node is called next.
    router,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        "retrieve": "rag",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [110]:
async def get_answer(inputs):
    messages = (await app.ainvoke(inputs))["messages"]
    response = messages[-1].content
    functions_called = len([message for message in messages if isinstance(message,FunctionMessage)])
    print(f"Response: {response}\nFunctions called: {functions_called}")

In [111]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
await get_answer(inputs)

content='' additional_kwargs={'function': {'arguments': '{"query": "San Francisco weather"}', 'name': 'duckduckgo_search'}}
content=' The latest weather information for San Francisco can be found by checking the forecasts from trusted sources such as Weather Underground, CBS San Francisco, and the Met Office. These sources provide current conditions, hourly and daily forecasts, radar, satellite images, maps, and more. Based on the context provided, it appears that there is a high chance of rain in San Francisco according to the National Weather Service forecast for Wednesday. However, for the most accurate and up-to-date information, I recommend visiting the websites or apps of these weather services.'
Response:  The latest weather information for San Francisco can be found by checking the forecasts from trusted sources such as Weather Underground, CBS San Francisco, and the Met Office. These sources provide current conditions, hourly and daily forecasts, radar, satellite images, maps,

In [112]:
inputs = {"messages": [HumanMessage(content="what is 2 + 5")]}
await get_answer(inputs)

content=' The sum of 2 and 5 is 7.'
Response:  The sum of 2 and 5 is 7.
Functions called: 0


In [113]:
inputs = {"messages": [HumanMessage(content="what is the capital of France")]}
await get_answer(inputs)

content=' The capital city of France is Paris.'
Response:  The capital city of France is Paris.
Functions called: 0


In [114]:
inputs = {"messages": [HumanMessage(content="Search events happening in Austin this weekend. Give response as a list.")]}
await get_answer(inputs)

content='' additional_kwargs={'function': {'arguments': '{"query": "events happening in Austin this weekend"}', 'name': 'duckduckgo_search'}}
content=" Based on the context provided, here is a list of events happening in Austin this weekend (February 2-4, 2023):\n1. If You're In Love: An Evening of Shoegaze & Dream-Pop ft. Lovelorn & Heartchaser at Hotel Vegas on Friday, February 2.\n2. Transit Method (Album Release) with Bat Lips and Megafauna at Hotel Vegas also on Friday, February 2.\n3. Hunt for Treasure at the French Legation Flea Market on Saturday, February 3. This new open-air vintage fair in East Austin offers about 25 vendors and an archaeology-themed talk and demonstration by the Texas Historical Commission.\n4. There are also mentions of Lunar New Year celebrations and Super Bowl parties, but no specific details or dates were provided in the context.\n5. Additionally, there are ongoing events and annual festivals like SXSW, ACL Music Festival, Austin Film Festival, Formula 

In [115]:
inputs = {"messages": [HumanMessage(content="What was Alice doing for the Umbrella Corporation?")]}
await get_answer(inputs)

content='' additional_kwargs={'function': {'arguments': '{"query": "What was Alice\'s role in the Umbrella Corporation?"}', 'name': 'duckduckgo_search'}}
content='' additional_kwargs={'function': {'arguments': '{"query": "What was Alice\'s role in the Umbrella Corporation?"}', 'name': 'duckduckgo_search'}}
content=' Based on the context provided, Alice is seeking revenge against the Umbrella Corporation and uncovering their secrets with the help of an ally. She is not currently employed by or holding a role within the Umbrella Corporation.'
Response:  Based on the context provided, Alice is seeking revenge against the Umbrella Corporation and uncovering their secrets with the help of an ally. She is not currently employed by or holding a role within the Umbrella Corporation.
Functions called: 2


In [116]:
inputs = {"messages": [HumanMessage(content="What is the premise of the movie Argylle (2024)")]}
await get_answer(inputs)

content='' additional_kwargs={'function': {'arguments': '{"query": "What is the premise of the movie Argylle (2024)?"}', 'name': 'duckduckgo_search'}}
content=' The movie "Argylle" (2024) follows the story of secret agent Argylle, played by Henry Cavill. In the beginning, he infiltrates a Greek-set club where he meets a woman named LaGrange (Dua Lipa). They engage in a dance before she escapes amid gunfire from bad guys. The film is described as a fun and highly stylized spy movie with a star-studded cast. However, there have been criticisms about certain elements of the movie being unsuccessful. There is also a post-credits scene.'
Response:  The movie "Argylle" (2024) follows the story of secret agent Argylle, played by Henry Cavill. In the beginning, he infiltrates a Greek-set club where he meets a woman named LaGrange (Dua Lipa). They engage in a dance before she escapes amid gunfire from bad guys. The film is described as a fun and highly stylized spy movie with a star-studded cas

In [118]:
inputs = {"messages": [HumanMessage(content="How did Argylle (2024) do on the box office?")]}
await get_answer(inputs)

content='' additional_kwargs={'function': {'arguments': '{"query": "How did Argylle (2024) do on the box office?"}', 'name': 'duckduckgo_search'}}
content=' The box office earnings for "Argylle (2024)" are reported to have been around $18 million during its opening weekend. However, it is considered a career-worst showing for director Matthew Vaughn.'
Response:  The box office earnings for "Argylle (2024)" are reported to have been around $18 million during its opening weekend. However, it is considered a career-worst showing for director Matthew Vaughn.
Functions called: 1


In [119]:
inputs = {"messages": [HumanMessage(content="Is Argylle (2024) a hit or a flop movie? Why?")]}
await get_answer(inputs)

content='' additional_kwargs={'function': {'arguments': '{"query": "Is Argylle (2024) a hit or a flop movie? Why?"}', 'name': 'duckduckgo_search'}}
content=' Based on the context provided, it appears that "Argylle (2024)" is currently considered a box office flop. The movie had a high budget of $200 million and failed to generate sufficient ticket sales during its theatrical release, with reported earnings of around $18 million. Additionally, the article mentions that Apple, the studio behind the film, has experienced its first box office failure with "Argylle."'
Response:  Based on the context provided, it appears that "Argylle (2024)" is currently considered a box office flop. The movie had a high budget of $200 million and failed to generate sufficient ticket sales during its theatrical release, with reported earnings of around $18 million. Additionally, the article mentions that Apple, the studio behind the film, has experienced its first box office failure with "Argylle."
Function

In [127]:
inputs = {"messages": [HumanMessage(content="Search for sample Langchain and Langgraph code in python")]}
await get_answer(inputs)

content=' I\'m unable to directly provide you with specific code samples for Langchain or Langgraph in Python as I don\'t have access to their exact implementations. However, if you search online using keywords like "Langchain Python code sample" or "Langgraph Python code example", you might find some resources that can help you get started. Good luck with your project!'
Response:  I'm unable to directly provide you with specific code samples for Langchain or Langgraph in Python as I don't have access to their exact implementations. However, if you search online using keywords like "Langchain Python code sample" or "Langgraph Python code example", you might find some resources that can help you get started. Good luck with your project!
Functions called: 0


In [124]:
inputs = {"messages": [HumanMessage(content="What did the president say about Ketanji Brown Jackson")]}
await get_answer(inputs)

content='' additional_kwargs={'function': {'arguments': '{"query": "What did the president say about Ketanji Brown Jackson"}', 'name': 'duckduckgo_search'}}
content=' In the context provided, former President Trump and President Biden have different views on Ketanji Brown Jackson\'s stance during a Supreme Court hearing. The article mentions that Justice Ketanji Brown Jackson, appointed by President Biden, seemed wary of arguments regarding the 14th amendment applying to presidents, as it does not specifically include the word "president" in the clause itself.'
Response:  In the context provided, former President Trump and President Biden have different views on Ketanji Brown Jackson's stance during a Supreme Court hearing. The article mentions that Justice Ketanji Brown Jackson, appointed by President Biden, seemed wary of arguments regarding the 14th amendment applying to presidents, as it does not specifically include the word "president" in the clause itself.
Functions called: 1
